In [1]:
#Import libraries
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import TavilySearchTool, FileWriterTool
from dotenv import load_dotenv
from IPython.display import display, Markdown

In [2]:
# Load environment variables (TAVILY_API_KEY, LLM_GATEWAY_URL, etc.)
load_dotenv(override=True)

True

In [3]:
# --- Configuration ---
# 1. Update the LLM initialization for CrewAI with TrueFoundry
llm = LLM(
    model=os.getenv("LLM_MODEL_CREWAI"),
    base_url=os.getenv("LLM_GATEWAY_URL"),
    api_key=os.getenv("TFY_API_KEY")
)

In [4]:
# --- Tools Setup ---
# Tavily Search Tool for web research
tavily_search_tool = TavilySearchTool()

# File Writer Tool for saving the final report
file_writer_tool = FileWriterTool()

In [5]:
# --- 2. Agents Definition ---
# 1) Researcher Agent (Uses TavilySearchTool)
researcher_agent = Agent(
    role="Senior Web Researcher",
    goal="Gather the latest and most relevant information about the user's query and format it as a comprehensive summary.",
    backstory="""You are an expert at utilizing the Tavily web search tool to find real-time, accurate, 
                 and cited information on any given topic. Your output is precise and well-structured.""",
    tools=[tavily_search_tool],
    llm=llm,
    verbose=True,
    memory=False,
    allow_delegation=False
)

In [6]:
# 2) Report Agent (Uses FileWriterTool)
report_agent = Agent(
    role="Professional Technical Writer",
    goal="Write a final, professionally formatted markdown report based on the context provided by the Researcher Agent",
    backstory="""You are a meticulous technical writer who turns raw research data into polished, 
                 production-ready documentation.""",
    tools=[],
    llm=llm,
    verbose=True,
)

In [7]:
# --- 3. Tasks Definition ---
research_task = Task(
    description="""Conduct an 'advanced' web search for the user's query: '{query}'. 
                   Focus on recent developments (past 6 months) and list all sources.
                   The final output must be a single, well-structured text summary of findings.""",
    expected_output="A comprehensive, cited summary of the research topic.",
    agent=researcher_agent,
)

In [8]:
report_task = Task(
    description="""Based on the summary provided by the Researcher Agent, write a final report.
                   The report must be in **Markdown format** with a title and bullet points.""",
    expected_output="A Markdown formatted report with a title and bullet points.",
    agent=report_agent,
    context=[research_task] # Task waits for research to complete
)

In [9]:
# --- Crew Initialization ---
research_crew = Crew(
    agents=[researcher_agent, report_agent],
    tasks=[research_task, report_task],
    process=Process.sequential,
    verbose=True # Higher verbosity for clear demonstration
)

In [10]:
# --- Function to be Wrapped by FastMCP --- this is synchronous here, but we'll make it asynchronous in the final script.
def run_research_analysis(query: str):
    """Executes the two-agent research crew."""
    print(f"--- Starting Crew for query: {query} ---")
    
    # Pass the user query to the crew's initial task
    result = research_crew.kickoff(inputs={"query": query})
    
    # Return the final output message from the Report Agent
    return result

In [11]:
# Example Hackathon Query
query = "Latest developments in quantum computing hardware in 2025" 

final_output = run_research_analysis(query)

print("\n\n--- Final Result from Crew ---")
print(final_output)

--- Starting Crew for query: Latest developments in quantum computing hardware in 2025 ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7e22edc9-2fc8-4665-9a4d-96a08d740279                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Web Researcher                                                                                   │
│                                                                                                                 │
│  Task: Conduct an 'advanced' web search for the user's query: 'Latest developments in quantum computing         │
│  hardware in 2025'.                                                                                             │
│                     Focus on recent developments (past 6 months) and list all sources.                          │
│                     The final output must be a single, well-structured text summary of findings.                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Web Researcher                                                                                   │
│                                                                                                                 │
│  Thought: Action: Tavily Search                                                                                 │
│                                                                                                                 │
│  Using Tool: Tavily Search                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional Technical Writer                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Quantum Computing Developments Report – 2025                                                                 │
│                                                                                                                 │
│  Recent developments in quantum computing hardware in 2025 highlight significant advancements and emerging      │
│  trends within the sector. Below are the key points:                                                            │
│                                                                                                                 │
│  - **Fujitsu and RIKEN's New Quantum Computer**                                                                 │
│    - In April 2025, Fujitsu and RIKEN unveiled a groundbreaking **256-qubit superconducting quantum             │
│  computer**.                                                                                                    │
│    - This new model is a remarkable enhancement from their previous model in 2023, which housed **64 qubits**.  │
│    - The advancements in this technology are expected to significantly accelerate quantum computing             │
│  capabilities, enabling more complex computations that were previously infeasible.                              │
│    - Source: SpinQ, 2025 [1].                                                                                   │
│                                                                                                                 │
│  - **Challenges and Innovations in Quantum Hardware**                                                           │
│    - Despite the advancements, the industry continues to face substantial challenges related to scalability.    │
│    - Insights published in **Nature Electronics** highlight that while progress is being made, achieving        │
│  practical applications remains a significant hurdle that requires attention.                                   │
│    - Source: Nature Electronics, 2025 [2].                                                                      │
│                                                                                                                 │
│  - **Role of Quantum Technologies Beyond Computing**                                                            │
│    - McKinsey's **Quantum Technology Monitor 2025** underscores the importance of quantum technologies not      │
│  just in computing, but also in sensing and communication.                                                      │
│    - The report suggests that 2025 is crucial for the integration of quantum technologies into practical        │
│  applications across various industries.                                                                        │
│    - Source: McKinsey, 2025 [3].                                                                                │
│                                                                                                                 │
│  - **D-Wave's Performance Breakthrough**                                                                        │
│    - D-Wave announced that its quantum computer has outperformed classical supercomputers for specific tasks.   │
│    - This significant milestone represents a shift in how certain computational challenges can be approached,   │
│  illustrating the potential of quantum computing techno

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0c496594-86b2-43c4-a0ff-c3d2d4d136b0                                                                     │
│  Agent: Professional Technical Writer                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7e22edc9-2fc8-4665-9a4d-96a08d740279                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Quantum Computing Developments Report – 2025                                                   │
│                                                                                                                 │
│  Recent developments in quantum computing hardware in 2025 highlight significant advancements and emerging      │
│  trends within the sector. Below are the key points:                                                            │
│                                                                                                                 │
│  - **Fujitsu and RIKEN's New Quantum Computer**                                                                 │
│    - In April 2025, Fujitsu and RIKEN unveiled a groundbreaking **256-qubit superconducting quantum             │
│  computer**.                                                                                                    │
│    - This new model is a remarkable enhancement from their previous model in 2023, which housed **64 qubits**.  │
│    - The advancements in this technology are expected to significantly accelerate quantum computing             │
│  capabilities, enabling more complex computations that were previously infeasible.                              │
│    - Source: SpinQ, 2025 [1].                                                                                   │
│                                                                                                                 │
│  - **Challenges and Innovations in Quantum Hardware**                                                           │
│    - Despite the advancements, the industry continues to face substantial challenges related to scalability.    │
│    - Insights published in **Nature Electronics** highlight that while progress is being made, achieving        │
│  practical applications remains a significant hurdle that requires attention.                                   │
│    - Source: Nature Electronics, 2025 [2].                                                                      │
│                                                                                                                 │
│  - **Role of Quantum Technologies Beyond Computing**                                                            │
│    - McKinsey's **Quantum Technology Monitor 2025** underscores the importance of quantum technologies not      │
│  just in computing, but also in sensing and communication.                                                      │
│    - The report suggests that 2025 is crucial for the integration of quantum technologies into practical        │
│  applications across various industries.                                                                        │
│    - Source: McKinsey, 2025 [3].                                                                                │
│                                                                                                                 │
│  - **D-Wave's Performance Breakthrough**                                                                        │
│    - D-Wave announced that its quantum computer has outperformed classical supercomputers for specific tasks.   │
│    - This significant milestone represents a shift in 

╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: 35e40f78-e2bc-47f0-9ae7-48a416357211                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/35e40f78-e2bc-47f0-9ae7-48a416357211?access_code=TRA │
│ CE-161ce3cfaf                                                                                                   │
│ 🔑 Access Code: TRACE-161ce3cfaf                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



--- Final Result from Crew ---
# Quantum Computing Developments Report – 2025

Recent developments in quantum computing hardware in 2025 highlight significant advancements and emerging trends within the sector. Below are the key points:

- **Fujitsu and RIKEN's New Quantum Computer**  
  - In April 2025, Fujitsu and RIKEN unveiled a groundbreaking **256-qubit superconducting quantum computer**.  
  - This new model is a remarkable enhancement from their previous model in 2023, which housed **64 qubits**.  
  - The advancements in this technology are expected to significantly accelerate quantum computing capabilities, enabling more complex computations that were previously infeasible.  
  - Source: SpinQ, 2025 [1].

- **Challenges and Innovations in Quantum Hardware**  
  - Despite the advancements, the industry continues to face substantial challenges related to scalability.  
  - Insights published in **Nature Electronics** highlight that while progress is being made, achieving prac

In [12]:
#Final response
display(Markdown(final_output.raw))

# Quantum Computing Developments Report – 2025

Recent developments in quantum computing hardware in 2025 highlight significant advancements and emerging trends within the sector. Below are the key points:

- **Fujitsu and RIKEN's New Quantum Computer**  
  - In April 2025, Fujitsu and RIKEN unveiled a groundbreaking **256-qubit superconducting quantum computer**.  
  - This new model is a remarkable enhancement from their previous model in 2023, which housed **64 qubits**.  
  - The advancements in this technology are expected to significantly accelerate quantum computing capabilities, enabling more complex computations that were previously infeasible.  
  - Source: SpinQ, 2025 [1].

- **Challenges and Innovations in Quantum Hardware**  
  - Despite the advancements, the industry continues to face substantial challenges related to scalability.  
  - Insights published in **Nature Electronics** highlight that while progress is being made, achieving practical applications remains a significant hurdle that requires attention.  
  - Source: Nature Electronics, 2025 [2].

- **Role of Quantum Technologies Beyond Computing**  
  - McKinsey's **Quantum Technology Monitor 2025** underscores the importance of quantum technologies not just in computing, but also in sensing and communication.  
  - The report suggests that 2025 is crucial for the integration of quantum technologies into practical applications across various industries.  
  - Source: McKinsey, 2025 [3].

- **D-Wave's Performance Breakthrough**  
  - D-Wave announced that its quantum computer has outperformed classical supercomputers for specific tasks.  
  - This significant milestone represents a shift in how certain computational challenges can be approached, illustrating the potential of quantum computing technology.  
  - Source: Constellation Research, 2025 [4].

- **Future Prospects of Quantum Computing**  
  - The year 2025 is recognized as a pivotal moment when quantum computing transitions from a concept to being firmly in the realm of practical applications.  
  - Researchers and industry leaders are optimistic that ongoing breakthroughs in quantum hardware will determine how swiftly quantum computing becomes a valuable tool for real-world applications.  
  - Source: SpinQ, 2025 [5].

Sources:  
[1] Quantum Computing News: ICQE 2025 & Latest Quantum Research - SpinQ  
[2] A qubit update | Nature Electronics  
[3] The Year of Quantum: From concept to reality in 2025 - McKinsey  
[4] 2025 is the year of quantum computing (already) - Constellation Research  
[5] Quantum Hardware Explained: A Complete Guide for 2025 - SpinQ